# Loading SpikeSorting Data

In [ ]:
# Turn off logging and disable tqdm this is a hidden cell on docs page
import logging
import os

logger = logging.getLogger('ibllib')
logger.setLevel(logging.CRITICAL)

os.environ["TQDM_DISABLE"] = "1"

Spikesorted output of electrophysiology data.

## Relevant Alf objects
* channels
* clusters
* spikes
* waveforms

## Loading

In [ ]:
from one.api import ONE
from brainbox.io.one import SpikeSortingLoader
one = ONE(base_url='https://openalyx.internationalbrainlab.org')

In [ ]:
pid = 'da8dfec1-d265-44e8-84ce-6ae9c109b8bd' 
ssl = SpikeSortingLoader(pid=pid, one=one)
spikes, clusters, channels = ssl.load_spike_sorting()
clusters = ssl.merge_clusters(spikes, clusters, channels)
waveforms = ssl.load_spike_sorting_object('waveforms')  # loads in the template waveforms

Alternatively, one can instantiate the spike sorting loader using the session unique identifier eid and the probe name pname:

In [ ]:
eid, pname = one.pid2eid(pid)
sl = SpikeSortingLoader(eid=eid, pname=pname, one=one)
spikes, clusters, channels = sl.load_spike_sorting()
clusters = sl.merge_clusters(spikes, clusters, channels)

## More details
* [Description of spikesorting datasets](https://docs.google.com/document/d/1OqIqqakPakHXRAwceYLwFY9gOrm8_P62XIfCTnHwstg/edit#heading=h.5houj8bng5o)
* Available spikesorting algorithms
    * [pykilosort](https://github.com/int-brain-lab/pykilosort#readme) (All data)
    * [Kilosort 2.5](https://github.com/MouseLand/Kilosort/releases/tag/v2.5) (Most data collected before August 2021)

## Important information
1. Data sorted with pykilosort is loaded by default. If the pykilosort spikesorting data is not available, the matlab kilosort 2.5 version will be loaded. See [Example 1](#Example-1:-Loading-different-spikesorting-versions) for more information.  
2. The channel locations in the brain can come from several sources. it will load the most advanced version of the histology available, regardless of the spike sorting version loaded.  
The steps, from most to least advanced, are:
    * **alf**: the final version of channel locations, same as resolved with the difference that data has been written out to files
    * **resolved**: channel location alignments have been agreed upon
    * **aligned**: channel locations have been aligned, but review or other alignments are pending, potentially not accurate
    * **traced**: the histology track has been recovered from microscopy, however the depths may not match, inacurate data
3. The attributes `mlapdv`, `atlas_ids` and `acronyms` in the `clusters` and `channels` objects are only available for probe insertions where `sl.histology` is equal to **traced**, **aligned**, **resolved** or **alf**.
4. The cluster and channel locations in the brain are only considered final for probe insertions with `sl.histology='resolved'`or `sl.histology='alf'`.

## Useful modules
* [brainbox.io.one.SpikeSortingLoader](https://int-brain-lab.github.io/iblenv/_autosummary/brainbox.io.one.html#brainbox.io.one.SpikeSortingLoader)
* [brainbox.ephys_plots](https://int-brain-lab.github.io/iblenv/_autosummary/brainbox.ephys_plots.html)
* [brainbox.population.decode.get_spike_counts_in_bins](https://int-brain-lab.github.io/iblenv/_autosummary/brainbox.population.decode.html#brainbox.population.decode.get_spike_counts_in_bins)
* [brainbox.processing](https://int-brain-lab.github.io/iblenv/_autosummary/brainbox.processing.html#brainbox.processing)
* [brainbox.metrics.single_units](https://int-brain-lab.github.io/iblenv/_autosummary/brainbox.metrics.single_units.html#module-brainbox.metrics.single_units)

## Exploring spikesorting data

### Example 1: Loading different spikesorting versions

In [ ]:
# By default, if available, the data spikesorted with pykilosort is loaded. 
# To find the spikesorting version that is loaded we can use
sl.collection

# To see all available spikesorted data for this probe insertion we can list the collections. 
# N.B. ks2.5 matlab spikesorted data is stored in the alf/probe00 folder
sl.collections

# The following can be used to load a specific version of spikesorting

# pykilosort version
spikes, clusters, channels = sl.load_spike_sorting(spike_sorter='pykilosort')

# ks2.5 matlab version
spikes, clusters, channels = sl.load_spike_sorting(spike_sorter='')

### Example 2: Loading additional data

In [ ]:
# The default spikes and cluster attributes loaded are:
# spikes - amps, clusters, depths, times
# cluster - channels, depths, metrics

#Other attributes can additionally be loaded in the following way
spikes, clusters, channels = sl.load_spike_sorting(dataset_types=['clusters.amps', 'spikes.samples'])
clusters = sl.merge_clusters(spikes, clusters, channels)

### Example 3: Compute firing rate across session

In [ ]:
from brainbox.ephys_plots import image_fr_plot
from iblutil.numerical import bincount2D
import numpy as np

time_bin = 0.05 # time bin in seconds
depth_bin = 10 # depth bin in um

# Remove any nan values
kp_idx = np.bitwise_and(~np.isnan(spikes['times']), ~np.isnan(spikes['depths']))

fr, time, depth = bincount2D(spikes['times'][kp_idx], spikes['depths'][kp_idx], time_bin, depth_bin)

### Example 4: Find clusters labelled as good

In [ ]:
good_clusterIDs = clusters['cluster_id'][clusters['label'] == 1]

### Example 5: Plot a raster for all units

In [ ]:
sl.raster(spikes, channels)

## Other relevant examples
* COMING SOON